# Project: mental health in Switzerland

## 1. dataset selection & analysis

### dataset 1

### data set 2

### conclusion

## 2. datset cleaning

explain that we began with pandas (local proof of concept) and scaled up using spark

### 2.1 unnesting the json

### 2.2 column selection

### 2.3 language filtering

### 2.4 sentiment analysis

### 2.5 spam removal

### 2.6 time format encoding

### 2.7 text treatment

lowercase, normalize (unicode)

url removal, RT removal

## 3. NLP methods

### 3.1. tokenizing

### 3.2 stop word removal

### 3.3 stemming

### 3.4 dictionary processing

#### building the dictionary

#### processing the dictionary

### 3.5 processing the data

## 4. ML 

### 4.1. labeling the tweets

### 4.2 construcing features TF-IDF

### 4.3 train SVM classifier

### 4.4 relabel training set

## 5. final data analysis

LDA to find similarities

## 6. Conclusions

# References and bibliography

[1] https://github.com/master/spark-stemming

http://nbviewer.jupyter.org/gist/mizvol/eb24770ac3d5d598463f972e2a669f03

https://spark.apache.org/docs/2.1.0/ml-features.html

http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/

https://www.rand.org/content/dam/rand/pubs/rgs_dissertations/RGSD300/RGSD391/RAND_RGSD391.pdf

https://www.aaai.org/ocs/index.php/ICWSM/ICWSM11/paper/viewFile/2880/3264

http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/